In [17]:
import fastf1
import pandas as pd
# import numpy as np

In [18]:
# Convert time to seconds
def time_to_seconds(df):
    df['Time'] = df['Time'].dt.total_seconds()

    # Winner time -> 0
    df.loc[df['Position'] == 1, 'Time'] = 0
    df.rename(columns={'Time': 'Gap to Winner'}, inplace=True)

    return df

### Retrieve data for regular races

In [19]:
# Create a list to store each race's DataFrame
races = []

# Iterate over the rounds raced in the 2025 season so far (5 - Next up: Miami GP)
for round_number in range(1, 6):
    # Load the race session
    session = fastf1.get_session(2025, round_number, 'R')
    session.load()

    # Select the relevant columns
    df = session.results[['DriverNumber', 'Abbreviation', 'TeamName', 'FullName', 'Position', 'GridPosition', 'Time', 'Points']].copy()

    # Create a new column for the driver's gain/loss in position
    df['PositionGainLoss'] = df['GridPosition'] - df['Position']

    # Add additional columns for race metadata
    df['RaceRound'] = round_number
    df['RaceName'] = session.event['EventName']

    df = time_to_seconds(df)

    # Append the DataFrame to the list
    races.append(df)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info


req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '23', '18', '27', '16', '81', '44', '10', '22', '31', '87', '30', '5', '14', '55', '7', '6']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.5.3]
req     

### Retrieve data for Sprint Races

In [20]:
sprints = ['China']

for round in sprints:
    # Load the race session
    session = fastf1.get_session(2025, round, 'S')
    session.load()

    # Select the relevant columns
    df = session.results[['DriverNumber', 'Abbreviation', 'TeamName', 'FullName', 'Position', 'GridPosition', 'Time', 'Points']].copy()

    # Create a new column for the driver's gain/loss in position
    df['PositionGainLoss'] = df['GridPosition'] - df['Position']

    # Add additional columns for race metadata
    df['RaceRound'] = round_number
    df['RaceName'] = session.event['EventName']

    df = time_to_seconds(df)

    # Append the DataFrame to the list
    races.append(df)

core           INFO 	Loading data for Chinese Grand Prix - Sprint [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 44 completed the race distance 00:00.020000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['44', '81', '1', '63', '16', '22', '12', '4', '18', '14', '23', 

In [21]:
final_df = pd.concat(races, ignore_index=True)

In [22]:
# Reorder the columns for better readability
column_order = ['RaceRound', 'RaceName', 'DriverNumber', 'Abbreviation', 'FullName', 'TeamName', 'Position', 'GridPosition', 'PositionGainLoss','Gap to Winner', 'Points']
final_df = final_df[column_order]

final_df

,RaceRound,RaceName,DriverNumber,Abbreviation,FullName,TeamName,Position,GridPosition,PositionGainLoss,Gap to Winner,Points
0,1,Australian Grand Prix,4,NOR,Lando Norris,McLaren,1.0,1.0,0.0,0.000,25.0
1,1,Australian Grand Prix,1,VER,Max Verstappen,Red Bull Racing,2.0,3.0,1.0,0.895,18.0
2,1,Australian Grand Prix,63,RUS,George Russell,Mercedes,3.0,4.0,1.0,8.481,15.0
3,1,Australian Grand Prix,12,ANT,Andrea Kimi Antonelli,Mercedes,4.0,16.0,12.0,10.135,12.0
4,1,Australian Grand Prix,23,ALB,Alexander Albon,Williams,5.0,6.0,1.0,12.773,10.0
...,...,...,...,...,...,...,...,...,...,...,...
115,5,Chinese Grand Prix,31,OCO,Esteban Ocon,Haas F1 Team,16.0,18.0,2.0,46.182,0.0
116,5,Chinese Grand Prix,55,SAI,Carlos Sainz,Williams,17.0,13.0,-4.0,51.376,0.0
117,5,Chinese Grand Prix,5,BOR,Gabriel Bortoleto,Kick Sauber,18.0,14.0,-4.0,53.940,0.0
118,5,Chinese Grand Prix,27,HUL,Nico Hulkenberg,Kick Sauber,19.0,20.0,1.0,56.682,0.0


In [ ]:
# Generate the CSV file with the results
# final_df.to_csv('f1_2025_season_results.csv', index=False)